Cameron Wright  
CSE4020 Sect. 01  
9th of July 2021  
<center> <h1>Final Project: DB Airways</h1> </center>  

***

## Table of Contents
═══════════════════════════════════════════════════════════════════════════════════════════════════
- [ Overview ](#overview)  
` ` 
- [ Formatting ](#formatting)  
` ` 
- [ Step A ](#a)  
    - [ Create database ](#database)
    - [ Create schema ](#crt_schema)
    - [ Insert schema ](#ins_schema)
    - [ Insert CSV data ](#csv)  
` `  
- [ Step B ](#b)  
` ` 
- [ Step C ](#c)  
` ` 
- [ Step D ](#d)  

<a name="overview"></a>
## Overview
═══════════════════════════════════════════════════════════════════════════════════════════════════
This project aims to create a mock database for an Airport owned by Mr.DB himself.  
Below are the actions completed to achieve this goal.   
***
__Those steps, in summary, are:__  
  
__A.)__ Create database & insert CSV files.  
__B.)__ Insert additional data into the database.  
__C.)__ Execute a query on the database.  
__D.)__ Create & test a procedure on the database.  

` `  
<a name="formatting"></a>
## Formatting
═══════════════════════════════════════════════════════════════════════════════════════════════════

For clarity, consistent formatting is used throughout this report.  
` `  

__Dividers:__
` ` 
- __Primary Sections__  
    ═══════════════  
` ` 

- __End of prints__  
    ╴╴╴╴╴╴╴╴╴╴╴  
` `  

__Labels:__  
- ## Primary Sections  
` ` 
- <h3> <ins> Subsections </ins> </h3>  
<br></br>
- *__Cell descriptions__*
` `  
` `  
Lastly, all images are stored locally in a scalable formats e.g., SVG.

` `  
<a name="a"></a>
## A.)
═══════════════════════════════════════════════════════════════════════════════════════════════════

<a name="database"></a>
<h3> <ins> Create DB Airways AWS Database </ins> </h3>

` `  
__*Connect to DB Airways AWS Instance*__

In [1]:
import mysql.connector

# RDS database credentials 
connection = mysql.connector.connect (
    host = "db-airways.cbpyvem0uhzn.us-east-1.rds.amazonaws.com",
    user = "admin",
    password = "password",
    database = "cse4020"
    )
connection.autocommit = True

# Attempting to connect to db-airways
cursor = connection.cursor(buffered=True)

# Check connection
if(connection):
    print("\x1b[32m" + "Connection successful" + "\x1b[0m\n" + str(connection))
else:
    print("\x1b[31m" + "Connection failed" + "\x1b[0m")
print('\n' + '╴' * 127)

Connection successful

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


<a name="crt_schema"></a>
<h3> <ins> Create schema from ER diagram </ins> </h3>  

__*Corresponding mySQL*__
```mysql
CREATE TABLE `person` (
  `name` varchar(20) PRIMARY KEY,
  `address` varchar(50),
  `phone` varchar(12)
);

CREATE TABLE `passenger` (
  `name` varchar(20),
  `ticket_number` varchar(6) PRIMARY KEY
);

CREATE TABLE `employee` (
  `name` varchar(20),
  `employee_number` int PRIMARY KEY,
  `salary` int
);

CREATE TABLE `pilot` (
  `flight_hours` int,
  `employee_number` int PRIMARY KEY
);

CREATE TABLE `booked_on` (
  `ticket_number` varchar(6) PRIMARY KEY,
  `flight_number` varchar(3)
);

CREATE TABLE `depature` (
  `date` date,
  `flight_number` varchar(3),
  PRIMARY KEY (`date`, `flight_number`)
);

CREATE TABLE `flight` (
  `flight_number` varchar(3) PRIMARY KEY,
  `origin` char(3),
  `destination` char(3),
  `depature_time` time,
  `arrival_time` time,
  `status` varchar(20)
);

CREATE TABLE `plane` (
  `maker` varchar(20),
  `model_number` char(4) PRIMARY KEY
);

CREATE TABLE `aircraft` (
  `serial_number` char(2),
  `model_number` char(4),
  PRIMARY KEY (`serial_number`, `model_number`)
);

CREATE TABLE `can_fly` (
  `employee_number` int,
  `model_number` char(4),
  PRIMARY KEY (`employee_number`, `model_number`)
);

ALTER TABLE `passenger` ADD FOREIGN KEY (`name`) REFERENCES `person` (`name`);

ALTER TABLE `employee` ADD FOREIGN KEY (`name`) REFERENCES `person` (`name`);

ALTER TABLE `passenger` ADD FOREIGN KEY (`ticket_number`) REFERENCES `booked_on` (`ticket_number`);

ALTER TABLE `pilot` ADD FOREIGN KEY (`employee_number`) REFERENCES `employee` (`employee_number`);

ALTER TABLE `aircraft` ADD FOREIGN KEY (`model_number`) REFERENCES `plane` (`model_number`);

ALTER TABLE `pilot` ADD FOREIGN KEY (`employee_number`) REFERENCES `can_fly` (`employee_number`);

ALTER TABLE `depature` ADD FOREIGN KEY (`flight_number`) REFERENCES `flight` (`flight_number`);

ALTER TABLE `booked_on` ADD FOREIGN KEY (`flight_number`) REFERENCES `depature` (`flight_number`);

ALTER TABLE `can_fly` ADD FOREIGN KEY (`model_number`) REFERENCES `plane` (`model_number`);
```

` `  
<a name="ins_schema"></a>
<h3> <ins> Implement schema from "DB_airport.sql" </ins> </h3>

In [2]:
# Drop all previous tables 
cursor.execute("DROP TABLE `aircraft`, `assigned_to`, `booked_on`, `can_fly`, `departure`, `employee`, `flight`, `passenger`, `person`, `pilot`, `plane`;")

In [3]:
# Read in file containing mySQL
file = open ("DB_airport.sql")
sql = file.read()

# Execute contained statements on database
print("\x1b[1mStatements executed:\x1b[0m\n")
for result in cursor.execute(sql, multi = True):
    if result.statement:
        print(result.statement)
    else:
        print("Rows affected: {}".format(result.rowcount))
print('\n' + '╴' * 127)

Statements executed:

CREATE TABLE `person` (
  `name` varchar(20) PRIMARY KEY,
  `address` varchar(50),
  `phone` varchar(12)
)
CREATE TABLE `passenger` (
  `name` varchar(20),
  `ticket_number` varchar(6) PRIMARY KEY
)
CREATE TABLE `employee` (
  `name` varchar(20),
  `salary` int,
  `employee_number` int PRIMARY KEY
)
CREATE TABLE `pilot` (
  `employee_number` int PRIMARY KEY,
  `flight_hours` int
)
CREATE TABLE `departure` (
  `date` date,
  `flight_number` varchar(3),
  PRIMARY KEY (`date`, `flight_number`)
)
CREATE TABLE `flight` (
  `flight_number` varchar(3) PRIMARY KEY,
  `origin` char(3),
  `destination` char(3),
  `depature_time` time,
  `arrival_time` time,
  `status` varchar(20)
)
CREATE TABLE `plane` (
  `maker` varchar(20),
  `model_number` char(4) PRIMARY KEY
)
CREATE TABLE `aircraft` (
  `serial_number` char(2),
  `model_number` char(4),
  PRIMARY KEY (`serial_number`, `model_number`)
)
CREATE TABLE `can_fly` (
  `employee_number` int,
  `model_number` char(4),
  PRIMA

` `  
__*Display all tables in the database*__

In [4]:
# Query for all tables in DB
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

table_names = []
print("\x1b[1mTables in DB:\x1b[0m")
for table in tables:
    # Create list of table names
    table_names.append(str(table).replace(",", "").replace("'", "").strip("()"))

# Display all table names
print('\n'.join(table_names) + '\n\n' + '╴' * 127)

Tables in DB:
aircraft
assigned_to
booked_on
can_fly
departure
employee
flight
passenger
person
pilot
plane

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


` `  
<a name="csv"></a>
<h3> <ins> Inserting CSV data </ins> </h3>

` `  
__*Function to get attributes from CSV file*__

In [5]:
def read_csv(file_name):
    records = []
    with open(file_name) as csv_file:
    
        # Cleaning attributes
        for line in csv_file:
            record = []
            line_token = line.split(",")
            for attribute in line_token:
                attribute = attribute.replace('"', "").strip()
                record.append(attribute)
            records.append(record)
    return records[1:]

` `  
__*Read in & display contents of CSV files*__

In [6]:
file_type = "{}.csv"

# Remove tables without csv's
no_csv = ["booked_on", "passenger", "assigned_to"]
for table in no_csv:
    if table in table_names: table_names.remove(table)
    
for table in table_names:
    csv_file = file_type.format(table)
    records = read_csv(csv_file)
    
    # Display each table & its attributes
    print("\x1b[1m\033[4m" + table + "\033[0m\x1b[0m")
    for record in records:
        print(*record)
    print()
print('╴' * 127)

aircraft
11 B727
13 B727
10 B747
13 B747
22 B757
93 B757
21 DC9
22 DC9
23 DC9
24 DC9
21 DC10
70 A310
80 A320

can_fly
1001 B727
1001 B747
1001 DC10
1002 DC9
1002 A340
1002 B757
1002 A320
1003 A310
1003 DC9

departure
June 30 2021 100
June 30 2021 112
June 30 2021 206
June 30 2021 334
June 30 2021 335
June 30 2021 337
June 30 2021 449
Jul 8 2021 100
Jul 8 2021 112
Jul 8 2021 206
Jul 8 2021 334
Jul 8 2021 395
Jul 8 2021 991

employee
Jones 50000.00 1001
Peters 45000.00 1002
Rowe 35000.00 1003
Phillips 25000.00 1004
Gates 5000000.00 1005
Clark 150000.00 1006
Warnock 500000.00 1007

flight
100 SLC BOS 08:00 17:50 on-time
206 DFW STL 09:00 11:40 on-time
334 ORD MIA 12:00 14:14 on-time
335 MIA ORD 15:00 17:14 on-time
336 ORD MIA 18:00 20:14 on-time
337 MIA ORD 20:30 23:53 on-time
121 STL SLC 07:00 09:13 on-time
122 STL YYV 08:30 10:19 on-time
330 JFK YYV 16:00 18:53 on-time
991 BOS ORD 17:00 18:22 on-time
394 DFW MIA 19:00 21:30 on-time
395 MIA DFW 21:00 23:43 on-time
449 CDG DEN 10:00 19:29

` `  
__*Execute querys to insert CSV data*__

In [7]:
# plane
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[7]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO plane VALUES('Boeing', 'B727');
INSERT INTO plane VALUES('Boeing', 'B747');
INSERT INTO plane VALUES('Boeing', 'B757');
INSERT INTO plane VALUES('MD', 'DC9');
INSERT INTO plane VALUES('MD', 'DC10');
INSERT INTO plane VALUES('Airbus', 'A310');
INSERT INTO plane VALUES('Airbus', 'A320');
INSERT INTO plane VALUES('Airbus', 'A330');
INSERT INTO plane VALUES('Airbus', 'A340');

Rows affected: 9

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [8]:
# aircraft
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[0]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO aircraft VALUES('11', 'B727');
INSERT INTO aircraft VALUES('13', 'B727');
INSERT INTO aircraft VALUES('10', 'B747');
INSERT INTO aircraft VALUES('13', 'B747');
INSERT INTO aircraft VALUES('22', 'B757');
INSERT INTO aircraft VALUES('93', 'B757');
INSERT INTO aircraft VALUES('21', 'DC9');
INSERT INTO aircraft VALUES('22', 'DC9');
INSERT INTO aircraft VALUES('23', 'DC9');
INSERT INTO aircraft VALUES('24', 'DC9');
INSERT INTO aircraft VALUES('21', 'DC10');
INSERT INTO aircraft VALUES('70', 'A310');
INSERT INTO aircraft VALUES('80', 'A320');

Rows affected: 13

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [9]:
# person
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[5]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO person VALUES('Smith', '123 Elm St.', '801-556-2239');
INSERT INTO person VALUES('Jones', '234 Oak St.', '801-552-2943');
INSERT INTO person VALUES('Peters', '345 Pine St.', '801-393-2230');
INSERT INTO person VALUES('Green', '435 Alder St.', '801-933-2320');
INSERT INTO person VALUES('Rowe', '348 Elder St.', '801-343-2320');
INSERT INTO person VALUES('Phillips', '395 Pine St.', '801-323-2320');
INSERT INTO person VALUES('Gates', '285 Kapok St.', '801-493-2203');
INSERT INTO person VALUES('Clark', '223 Easy St.', '801-193-2320');
INSERT INTO person VALUES('Warnock', '775 Main St.', '801-303-2222');
INSERT INTO person VALUES('Hooper', '456 Maple St.', '313-912-2101');
INSERT INTO person VALUES('Edwards', '567 Spruce St.', '801-228-6729');
INSERT INTO person VALUES('Majeris', '678 Willow St.', 'null');
INSERT INTO person VALUES('MacBride', '789 Fir St.', 'null');

Rows affected: 13

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴

In [10]:
# employee
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[3]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2] + "');"))
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)



Queries
INSERT INTO employee VALUES('Jones', '50000.00', '1001');
INSERT INTO employee VALUES('Peters', '45000.00', '1002');
INSERT INTO employee VALUES('Rowe', '35000.00', '1003');
INSERT INTO employee VALUES('Phillips', '25000.00', '1004');
INSERT INTO employee VALUES('Gates', '5000000.00', '1005');
INSERT INTO employee VALUES('Clark', '150000.00', '1006');
INSERT INTO employee VALUES('Warnock', '500000.00', '1007');

Rows affected: 7

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [11]:
# can_fly
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[1]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO can_fly VALUES('1001', 'B727');
INSERT INTO can_fly VALUES('1001', 'B747');
INSERT INTO can_fly VALUES('1001', 'DC10');
INSERT INTO can_fly VALUES('1002', 'DC9');
INSERT INTO can_fly VALUES('1002', 'A340');
INSERT INTO can_fly VALUES('1002', 'B757');
INSERT INTO can_fly VALUES('1002', 'A320');
INSERT INTO can_fly VALUES('1003', 'A310');
INSERT INTO can_fly VALUES('1003', 'DC9');

Rows affected: 9

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [12]:
# pilot
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[6]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO pilot VALUES('1001', '6000');
INSERT INTO pilot VALUES('1002', '24000');
INSERT INTO pilot VALUES('1003', '15000');

Rows affected: 3

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [13]:
# flight
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[4]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2]) + "', '" + str(record[3]) + "', '" + str(record[4]) + "', '" + str(record[5]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "', '" + str(record[2]) + "', '" + str(record[3]) + "', '" + str(record[4]) + "', '" + str(record[5]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO flight VALUES('100', 'SLC', 'BOS', '08:00', '17:50', 'on-time');
INSERT INTO flight VALUES('206', 'DFW', 'STL', '09:00', '11:40', 'on-time');
INSERT INTO flight VALUES('334', 'ORD', 'MIA', '12:00', '14:14', 'on-time');
INSERT INTO flight VALUES('335', 'MIA', 'ORD', '15:00', '17:14', 'on-time');
INSERT INTO flight VALUES('336', 'ORD', 'MIA', '18:00', '20:14', 'on-time');
INSERT INTO flight VALUES('337', 'MIA', 'ORD', '20:30', '23:53', 'on-time');
INSERT INTO flight VALUES('121', 'STL', 'SLC', '07:00', '09:13', 'on-time');
INSERT INTO flight VALUES('122', 'STL', 'YYV', '08:30', '10:19', 'on-time');
INSERT INTO flight VALUES('330', 'JFK', 'YYV', '16:00', '18:53', 'on-time');
INSERT INTO flight VALUES('991', 'BOS', 'ORD', '17:00', '18:22', 'on-time');
INSERT INTO flight VALUES('394', 'DFW', 'MIA', '19:00', '21:30', 'on-time');
INSERT INTO flight VALUES('395', 'MIA', 'DFW', '21:00', '23:43', 'on-time');
INSERT INTO flight VALUES('449', 'CDG', 'DEN', '10:00', '19:29', 'on

In [14]:
# departure
rows_affected = 0
csv_file = file_type.format(table)

table = table_names[2]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT INTO " + table + " VALUES(" + " STR_TO_DATE('" + str(record[0]) + " " + str(record[1]) + "', '%M %d %Y')" + ", '" + str(record[2]) + "');")
    cursor.execute("INSERT INTO " + table + " VALUES(" + " STR_TO_DATE('" + str(record[0]) + " " + str(record[1]) + "', '%M %d %Y')" + ", '" + str(record[2]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '100');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '112');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '206');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '334');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '335');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '337');
INSERT INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '449');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021', '%M %d %Y'), '100');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021', '%M %d %Y'), '112');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021', '%M %d %Y'), '206');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021', '%M %d %Y'), '334');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021', '%M %d %Y'), '395');
INSERT INTO departure VALUES( STR_TO_DATE('Jul 8 2021'

` `  
__*Commit changes to database*__

In [15]:
connection.commit()

` `  
__*Display all tables (in table_names) from the database*__

In [16]:
# For each table in table_names select all & then print
for name in table_names:
    print("\x1b[1m\033[4m" + name + "\033[0m\x1b[0m")
    cursor.execute("SELECT * FROM " + name + ";")
    for row in cursor.fetchall():
        print(*row)
    print()
print('╴' * 127)

aircraft
70 A310
80 A320
11 B727
13 B727
10 B747
13 B747
22 B757
93 B757
21 DC10
21 DC9
22 DC9
23 DC9
24 DC9

can_fly
1003 A310
1002 A320
1002 A340
1001 B727
1001 B747
1002 B757
1001 DC10
1002 DC9
1003 DC9

departure
2021-06-30 100
2021-07-08 100
2021-06-30 112
2021-07-08 112
2021-06-30 206
2021-07-08 206
2021-06-30 334
2021-07-08 334
2021-06-30 335
2021-06-30 337
2021-07-08 395
2021-06-30 449
2021-07-08 991

employee
Jones 50000 1001
Peters 45000 1002
Rowe 35000 1003
Phillips 25000 1004
Gates 5000000 1005
Clark 150000 1006
Warnock 500000 1007

flight
100 SLC BOS 8:00:00 17:50:00 on-time
112 DCA DEN 14:00:00 18:07:00 on-time
121 STL SLC 7:00:00 9:13:00 on-time
122 STL YYV 8:30:00 10:19:00 on-time
206 DFW STL 9:00:00 11:40:00 on-time
330 JFK YYV 16:00:00 18:53:00 on-time
334 ORD MIA 12:00:00 14:14:00 on-time
335 MIA ORD 15:00:00 17:14:00 on-time
336 ORD MIA 18:00:00 20:14:00 on-time
337 MIA ORD 20:30:00 23:53:00 on-time
394 DFW MIA 19:00:00 21:30:00 on-time
395 MIA DFW 21:00:00 23:43:00

` `  
## B.)
═══════════════════════════════════════════════════════════════════════════════════════════

` `  
__*Execute querys to insert CSV data from files*__

In [17]:
# booked_on
rows_affected = 0
csv_file = file_type.format(table)

table = no_csv[0]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT IGNORE INTO booked_on VALUES('DBA001', '100');
INSERT IGNORE INTO booked_on VALUES('DBA002', '206');
INSERT IGNORE INTO booked_on VALUES('DBA003', '334');
INSERT IGNORE INTO booked_on VALUES('DBA004', '449');
INSERT IGNORE INTO booked_on VALUES('DBA005', '991');
INSERT IGNORE INTO booked_on VALUES('DBA006', '991');
INSERT IGNORE INTO booked_on VALUES('DBA007', '100');
INSERT IGNORE INTO booked_on VALUES('DBA008', '100');
INSERT IGNORE INTO booked_on VALUES('DBA009', '449');
INSERT IGNORE INTO booked_on VALUES('DBA010', '449');
INSERT IGNORE INTO booked_on VALUES('DBA011', '991');
INSERT IGNORE INTO booked_on VALUES('DBA012', '100');

Rows affected: 12

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [18]:
# passenger
rows_affected = 0
csv_file = file_type.format(table)

table = no_csv[1]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT IGNORE INTO passenger VALUES('Smith', 'DBA001');
INSERT IGNORE INTO passenger VALUES('Green', 'DBA002');
INSERT IGNORE INTO passenger VALUES('Hooper', 'DBA003');
INSERT IGNORE INTO passenger VALUES('Edwards', 'DBA004');
INSERT IGNORE INTO passenger VALUES('MacBride', 'DBA005');
INSERT IGNORE INTO passenger VALUES('Gates', 'DBA006');
INSERT IGNORE INTO passenger VALUES('Rowe', 'DBA007');
INSERT IGNORE INTO passenger VALUES('Clark', 'DBA008');
INSERT IGNORE INTO passenger VALUES('Phillips', 'DBA009');
INSERT IGNORE INTO passenger VALUES('Warnock', 'DBA010');
INSERT IGNORE INTO passenger VALUES('Smith', 'DBA011');
INSERT IGNORE INTO passenger VALUES('Peters', 'DBA012');

Rows affected: 12

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


In [19]:
# more_departure
rows_affected = 0
csv_file = file_type.format(table)

csv_file = file_type.format("departure")
records = read_csv("more_departure.csv")


print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT IGNORE INTO departure VALUES(" + " STR_TO_DATE('" + str(record[0]) + " " + str(record[1]) + "', '%M %d %Y')" + ", '" + str(record[2]) + "');")
    cursor.execute("INSERT IGNORE INTO departure VALUES(" + " STR_TO_DATE('" + str(record[0]) + " " + str(record[1]) + "', '%M %d %Y')" + ", '" + str(record[2]) + "');")
    
rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '100');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '206');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '334');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '449');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('July 8 2021', '%M %d %Y'), '991');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('July 8 2021', '%M %d %Y'), '991');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('July 8 2021', '%M %d %Y'), '100');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('July 8 2021', '%M %d %Y'), '100');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '449');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('June 30 2021', '%M %d %Y'), '449');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('July 8 2021', '%M %d %Y'), '991');
INSERT IGNORE INTO departure VALUES( STR_TO_DATE('J

*No rows should be affected as all departure attributes are duplicates to those already in database.*  
` `  

In [20]:
# assigned_to
rows_affected = 0
csv_file = file_type.format(table)

table = no_csv[2]
csv_file = file_type.format(table)
records = read_csv(csv_file)

print("\x1b[1m\033[4mQueries\033[0m\x1b[0m")
for record in records:
    print("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
    cursor.execute("INSERT IGNORE INTO " + table + " VALUES('" + str(record[0]) + "', '" + str(record[1]) + "');")
rows_affected += cursor.rowcount
print("\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Queries
INSERT IGNORE INTO assigned_to VALUES('100', '1001');
INSERT IGNORE INTO assigned_to VALUES('100', '1002');
INSERT IGNORE INTO assigned_to VALUES('100', '1003');
INSERT IGNORE INTO assigned_to VALUES('100', '1004');
INSERT IGNORE INTO assigned_to VALUES('206', '1007');
INSERT IGNORE INTO assigned_to VALUES('337', '1003');
INSERT IGNORE INTO assigned_to VALUES('337', '1004');
INSERT IGNORE INTO assigned_to VALUES('337', '1005');
INSERT IGNORE INTO assigned_to VALUES('337', '1006');
INSERT IGNORE INTO assigned_to VALUES('100', '1001');
INSERT IGNORE INTO assigned_to VALUES('100', '1002');
INSERT IGNORE INTO assigned_to VALUES('991', '1006');
INSERT IGNORE INTO assigned_to VALUES('991', '1007');
INSERT IGNORE INTO assigned_to VALUES('112', '1007');

Rows affected: 1

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


` `  
__*Commit changes to database*__

In [21]:
connection.commit()

` `  
__*Display all tables in the database*__

In [22]:
# For each table in table_names & no_csv select all & then print
for name in (table_names + no_csv):
    print("\x1b[1m\033[4m" + name + "\033[0m\x1b[0m")
    cursor.execute("SELECT * FROM " + name + ";")
    for row in cursor.fetchall():
        print(*row)
    print()
print('╴' * 127)

aircraft
70 A310
80 A320
11 B727
13 B727
10 B747
13 B747
22 B757
93 B757
21 DC10
21 DC9
22 DC9
23 DC9
24 DC9

can_fly
1003 A310
1002 A320
1002 A340
1001 B727
1001 B747
1002 B757
1001 DC10
1002 DC9
1003 DC9

departure
2021-06-30 100
2021-07-08 100
2021-06-30 112
2021-07-08 112
2021-06-30 206
2021-07-08 206
2021-06-30 334
2021-07-08 334
2021-06-30 335
2021-06-30 337
2021-07-08 395
2021-06-30 449
2021-07-08 991

employee
Jones 50000 1001
Peters 45000 1002
Rowe 35000 1003
Phillips 25000 1004
Gates 5000000 1005
Clark 150000 1006
Warnock 500000 1007

flight
100 SLC BOS 8:00:00 17:50:00 on-time
112 DCA DEN 14:00:00 18:07:00 on-time
121 STL SLC 7:00:00 9:13:00 on-time
122 STL YYV 8:30:00 10:19:00 on-time
206 DFW STL 9:00:00 11:40:00 on-time
330 JFK YYV 16:00:00 18:53:00 on-time
334 ORD MIA 12:00:00 14:14:00 on-time
335 MIA ORD 15:00:00 17:14:00 on-time
336 ORD MIA 18:00:00 20:14:00 on-time
337 MIA ORD 20:30:00 23:53:00 on-time
394 DFW MIA 19:00:00 21:30:00 on-time
395 MIA DFW 21:00:00 23:43:00

` `  
## C.)
══════════════════════════════════════════════════════════════════════════════════════════

__*Query to find name, ticket_number, address, & phone of all people on flight 991*__

```mysql 
SELECT person.name, passenger.ticket_number, person.address, person.phone
FROM person, passenger, booked_on
WHERE person.name = passenger.name AND passenger.ticket_number = booked_on.ticket_number AND booked_on.flight_number = "991";
```

__*Execute query and display results*__

In [23]:
rows_affected = 0

cursor.execute("""
SELECT person.name, passenger.ticket_number, person.address, person.phone
FROM person, passenger, booked_on
WHERE person.name = passenger.name 
      AND passenger.ticket_number = booked_on.ticket_number 
      AND booked_on.flight_number = "991";
""")

print("\x1b[1m\033[4m│{:^15}│{:^15}│{:^15}│{:^15}│\033[0m\x1b[0m".format("passenger_name","ticket_number"," address","phone"))
for row in cursor.fetchall():
    print("│{:<15}│{:<15}│{:<15}│{:<15}│".format(row[0],row[1],row[2],row[3]))
rows_affected += cursor.rowcount
print("\n\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

│passenger_name │ ticket_number │    address    │     phone     │
│MacBride       │DBA005         │789 Fir St.    │null           │
│Gates          │DBA006         │285 Kapok St.  │801-493-2203   │
│Smith          │DBA011         │123 Elm St.    │801-556-2239   │


Rows affected: 3

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


*The query returned the correct results!*

` `  
## D.)
══════════════════════════════════════════════════════════════════════════════════════════

__*Stored procedure to approximate years pilot has been flying*__

```mysql 
CREATE PROCEDURE estimate_pilot_years(employee_number INT)
BEGIN
    /* Hours per year (75*12) */
    DECLARE annual INT DEFAULT 900;
    /* Number of estimated years */ 
	DECLARE years float(9,2) DEFAULT 0;
    
	SET years = ((SELECT pilot.flight_hours 
                    FROM pilot
                    WHERE pilot.employee_number = employee_number)/annual);
    SELECT years;
    
END;
```

In [24]:
cursor.execute("DROP procedure IF EXISTS estimate_pilot_years")

In [25]:
rows_affected = 0

cursor.execute("""
CREATE PROCEDURE estimate_pilot_years(employee_number INT)
BEGIN
    /* Hours per year (75*12) */
    DECLARE annual INT DEFAULT 900;
    /* Number of estimated years */ 
    DECLARE years float(9,2) DEFAULT 0;
    
    SET years = ((SELECT pilot.flight_hours 
                    FROM pilot
                    WHERE pilot.employee_number = employee_number)/annual);
    SELECT years;
    
END;
""")

rows_affected += cursor.rowcount
print("\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

Rows affected: 0

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


` `  
__*Get all pilot employee numbers*__

In [26]:
rows_affected = 0

cursor.execute("""
SELECT *
FROM pilot;
""")

pilot_ids = []
pilot_hours = []
print("\x1b[1m\033[4m│{:^25}│\033[0m\x1b[0m".format("pilot.employee_number"))

for row in cursor.fetchall():
    print("│{:<25}│".format(row[0]))
    pilot_ids.append(row[0])
    pilot_hours.append(row[1])

rows_affected += cursor.rowcount
print("\n\n\x1b[1m" + "Rows affected: " + "\x1b[0m" + str(rows_affected) + "\n\n" + '╴' * 127)

│  pilot.employee_number  │
│1001                     │
│1002                     │
│1003                     │


Rows affected: 3

╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴╴


` `  
__*Call procedure for each id & display results*__

In [27]:
print("\x1b[1m\033[4m│{:^20}│{:^20}│{:^20}│\033[0m\x1b[0m".format("pilot ID", "flight_hours", "years_flown"))

for id in pilot_ids:
    cursor.callproc("estimate_pilot_years", [str(id)])
    result = [r.fetchall() for r in cursor.stored_results()]
    print("│{:<20}│{:<20}│{:<20}│".format(id, pilot_hours[pilot_ids.index(id)] , *result[0][0]))
    

│      pilot ID      │    flight_hours    │    years_flown     │
│1001                │6000                │6.67                │
│1002                │24000               │26.67               │
│1003                │15000               │16.67               │


In [28]:
# Close socket to database
cursor.close()

True